# Ekstraksi GLCM

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import math
import pandas as pd
import imutils.paths as path
from tqdm import tqdm
from glob import glob

In [ ]:
PATH = glob("filtered_data_fix/*/*.jpg")
PATH[:10]

## Convert to GrayScale & Resize 128x128

In [ ]:
data = []
for i in tqdm(PATH, desc="load"):
    fileImage = cv.imread(i)
    img = cv.cvtColor(fileImage, cv.COLOR_BGR2GRAY)
    a = cv.resize(img, (128, 128))
    data.append(a)
data[:5]

In [ ]:
def derajat0(img):
    max = np.max(img)
    imgTmp = np.zeros([max+1, max+1])
    for i in range(len(img)):
        for j in range(len(img[i])-1):
            imgTmp[img[i,j], img[i, j+1]] += 1
    
    transpos = np.transpose(imgTmp)
    data = imgTmp+transpos
    
    tmp = 0
    for i in range(len(data)):
        for j in range(len(data)):
            tmp+=data[i, j]
            
    for i in range(len(data)):
        for j in range(len(data)):
            data[i, j]/=tmp
    
    return data

In [ ]:
def derajat45(img):
    max = np.max(img)
    imgTmp = np.zeros([max+1, max+1])
    for i in range(len(img)-1):
        for j in range(len(img[i])-1):
            imgTmp[img[i+1,j], img[i, j+1]] += 1
    
    transpos = np.transpose(imgTmp)
    data = imgTmp+transpos
    
    tmp = 0
    for i in range(len(data)):
        for j in range(len(data)):
            tmp+=data[i, j]
            
    for i in range(len(data)):
        for j in range(len(data)):
            data[i, j]/=tmp
    
    return data

In [ ]:
def derajat90(img):
    max = np.max(img)
    imgTmp = np.zeros([max+1, max+1])
    for i in range(len(img)-1):
        for j in range(len(img[i])):
            imgTmp[img[i+1,j], img[i, j]] += 1
    
    transpos = np.transpose(imgTmp)
    data = imgTmp+transpos
    
    tmp = 0
    for i in range(len(data)):
        for j in range(len(data)):
            tmp+=data[i, j]
            
    for i in range(len(data)):
        for j in range(len(data)):
            data[i, j]/=tmp
    
    return data

In [ ]:
def derajat135(img):
    max = np.max(img)
    imgTmp = np.zeros([max+1, max+1])
    for i in range(len(img)-1):
        for j in range(len(img[i])-1):
            imgTmp[img[i,j], img[i+1, j+1]] += 1
    
    transpos = np.transpose(imgTmp)
    data = imgTmp+transpos
    
    tmp = 0
    for i in range(len(data)):
        for j in range(len(data)):
            tmp+=data[i, j]
            
    for i in range(len(data)):
        for j in range(len(data)):
            data[i, j]/=tmp
    
    return data

In [ ]:
hasil = []
for i in tqdm(range(len(data)), desc='GLCM'):
    dat = []
    dat.append(derajat0(data[i]))
    dat.append(derajat45(data[i]))
    dat.append(derajat90(data[i]))
    dat.append(derajat135(data[i]))
    hasil.append(dat)

In [ ]:
def contras(data):
    contras = 0
    for i in range(len(data)):
        for j in range(len(data)):
            contras += data[i,j]*pow(i-j,2)
    return contras

In [ ]:
def entropy(data):
    entro = 0
    for i in range(len(data)):
        for j in range(len(data)):
            if data[i,j] > 0.0:
                entro += -(data[i,j] * math.log(data[i,j]))
    return entro

In [ ]:
def contras(data):
    contras = 0
    for i in range(len(data)):
        for j in range(len(data)):
            contras += data[i,j]*pow(i-j,2)
    return contras

In [ ]:
def homogentitas(data):
    homogen = 0
    for i in range(len(data)):
        for j in range(len(data)):
            homogen += data[i,j]*(1+(pow(i-j,2)))
    return homogen

In [ ]:
def energy(data):
    ener = 0
    for i in range(len(data)):
        for j in range(len(data)):
            ener += data[i,j]**2
    return ener

In [ ]:
hasilnya = []

for j in tqdm(range(len(hasil)),desc='Ekstraksi'):
    da=[]
    da.append(PATH[j])
    for i in hasil[j]:
        dx = energy(i)
        da.append(dx)
        
        dh = homogentitas(i)
        da.append(dh)
        
        den = entropy(i)
        da.append(den)
        
        dco = contras(i)
        da.append(dco)
    
    da.append(str(PATH[j].split("\\")[1]))
    hasilnya.append(da)

In [ ]:
namatabel = ['file', 'energy_0', 'homogenity_0', 'entrophy_0', 'contras_0', 'energy_45', 'homogenity_45', 'entrophy_45', 'contras_45', 'energy_90', 'homogenity_90', 'entrophy_90', 'contras_90', 'energy_135', 'homogenity_135', 'entrophy_135', 'contras_135', 'Target']
df = pd.DataFrame(hasilnya, columns=namatabel)

In [ ]:
df.head()
df.shape

In [ ]:
df

In [ ]:
df.to_csv(r'data_filtered_fix.csv', index=False)

# Klasifikasi KNN

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from jcopml.utils import save_model, load_model

In [2]:
df = pd.read_csv("data_filtered_fix.csv")
df.head()

,file,energy_0,homogenity_0,entrophy_0,contras_0,energy_45,homogenity_45,entrophy_45,contras_45,energy_90,homogenity_90,entrophy_90,contras_90,energy_135,homogenity_135,entrophy_135,contras_135,Target
0,filtered_data_fix\Blight\Corn_Blight (1009).JPG,0.000259,195.767409,8.587177,194.767409,0.000245,208.004092,8.627077,207.004092,0.000324,107.079786,8.350959,106.079786,0.000257,233.384897,8.606781,232.384897,Blight
1,filtered_data_fix\Blight\Corn_Blight (1014).JPG,0.000320,260.168123,8.406391,259.168123,0.000316,260.337405,8.411763,259.337405,0.000385,173.546937,8.239666,172.546937,0.000299,302.560357,8.460982,301.560357,Blight
2,filtered_data_fix\Blight\Corn_Blight (1016).JPG,0.000355,197.739173,8.345707,196.739173,0.000321,236.215822,8.417721,235.215822,0.000418,103.358329,8.117015,102.358329,0.000335,192.313039,8.357372,191.313039,Blight
3,filtered_data_fix\Blight\Corn_Blight (1021).JPG,0.000343,242.821358,8.336452,241.821358,0.000322,286.307893,8.392211,285.307893,0.000461,126.721580,8.071687,125.721580,0.000349,243.099386,8.335640,242.099386,Blight
4,filtered_data_fix\Blight\Corn_Blight (1026).JPG,0.000326,241.402128,8.361688,240.402128,0.000320,246.716411,8.383937,245.716411,0.000436,99.706816,8.065426,98.706816,0.000315,256.822122,8.394468,255.822122,Blight


In [3]:
X = df.drop(columns=["Target","file"])
y = df.Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((960, 16), (240, 16), (960,), (240,))

In [4]:
# scaler = MinMaxScaler()
# scaler.fit(X_train)

# X_train_scaled = scaler.transform(X_train)
# X_test_scaled = scaler.transform(X_test)

In [5]:
# neighbor = range(1, 51, 2)
# train_score = []
# test_score = []

# for k in neighbor:
#     knn = KNeighborsClassifier(n_neighbors=k)
#     knn.fit(X_train_scaled, y_train)
    
#     train_score.append(knn.score(X_train_scaled, y_train))
#     test_score.append(knn.score(X_test_scaled, y_test))
    
# plt.plot(neighbor, train_score, label="train")
# plt.plot(neighbor, test_score, label="test")
# plt.legend()

# print(f"Max test score: {np.max(test_score)}")
# print(f"n_neighbor: {neighbor[np.argmax(test_score)]}")

# knn.score(X_train_scaled, y_train), knn.score(X_test_scaled, y_test)

In [6]:
# preds = knn.predict(X_test)
# y_test
# preds


## Preprocessor

In [7]:
num_pipe = Pipeline([
    ("scaler", MinMaxScaler())
])
preprocessor = ColumnTransformer([
    ('numeric', num_pipe, X_train.columns)
])
pipeline = Pipeline([
    ("prep", preprocessor),
    ("algo", KNeighborsClassifier())
])

In [8]:
parameter = {
    "algo__n_neighbors": range(1, 51, 2),
    "algo__weights": ["uniform", "distance"],
#     "algo__p": [1, 2]
}

model = GridSearchCV(pipeline, parameter, cv=4, n_jobs=-1, verbose=1)
model.fit(X_train, y_train)

Fitting 4 folds for each of 50 candidates, totalling 200 fits


GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('prep',
                                        ColumnTransformer(transformers=[('numeric',
                                                                         Pipeline(steps=[('scaler',
                                                                                          MinMaxScaler())]),
                                                                         Index(['energy_0', 'homogenity_0', 'entrophy_0', 'contras_0', 'energy_45',
       'homogenity_45', 'entrophy_45', 'contras_45', 'energy_90',
       'homogenity_90', 'entrophy_90', 'contras_90', 'energy_135',
       'homogenity_135', 'entrophy_135', 'contras_135'],
      dtype='object'))])),
                                       ('algo', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid={'algo__n_neighbors': range(1, 51, 2),
                         'algo__weights': ['uniform', 'distance']},
             verbose=1)

In [9]:
model.best_params_

{'algo__n_neighbors': 9, 'algo__weights': 'distance'}

In [10]:
model.score(X_train, y_train), model.score(X_test, y_test)

(1.0, 0.825)

In [11]:
preds = model.predict(X_test)
acc = (y_test == preds).astype(int)
acc.mean()
acc[:20]

235     1
841     1
939     1
334     1
1060    1
188     1
472     1
546     0
134     1
740     1
545     1
909     1
427     1
532     1
872     0
781     0
731     1
669     1
270     1
87      1
Name: Target, dtype: int32